In [334]:
from plotnine import *
import pandas
from sklearn import preprocessing
import itertools
from statistics import mean
from solarized import *
import tikzplotlib
pandas.options.display.max_rows=10

Index(['index', 'bytes_memory', 'cpu_percent', 'bytes_sent', 'bytes_recv',
       'wall_time', 'exitcode', 'db_time', 'rep', 'net', 'scale', 'wflow',
       'optimizer'],
      dtype='object')

In [ ]:
# 1.{5end_to_end} tpch 1,2,3,4,5 with scale 1, 10
# 2 {5module4} and 3. module4 | also {5module4mem} and {5module4net}
# 3.{5tpchmodin} tpch 1, 4, 5 with modin on scale 10
# 4.{5micro_scales} micros with scale 1, 10 | also {5micro_traffic}
# 5.{5micro_net} micro join, selection with net=wan and scale 1

In [ ]:
factors= ['wflow', 'optimizer', 'net', 'scale']
df=pandas.read_feather('specific-reposts/5end_to_end.feather')
df.columns

# CHECK

In [335]:
df['exitcode'].value_counts()

0    186285
1     21940
Name: exitcode, dtype: int64

In [336]:
df.query("exitcode == 1")['wflow'].unique()

array(['tpch1.py', 'q07.sql', 'q08.sql', 'q09.sql'], dtype=object)

# PREPROCESS

In [399]:
overview = df.groupby(by=factors).first().reset_index()
overview.columns

Index(['wflow', 'optimizer', 'net', 'scale', 'index', 'bytes_sent',
       'wall_time', 'db_time', 'gb_memory', 'cpu', 'gb_net',
       'procedural_time'],
      dtype='object')

In [ ]:
overview = overview["wflow", "optimizer", "scale", "shared_wall_time"]]

# PROCEDURAL AND DB TIME

In [402]:
def prep_for_plot(origin_df: pandas.DataFrame, measurements:list):
    prj = origin_df[factors+measurements]
    return prj.melt(id_vars=factors, value_vars=measurements,
                    value_name='measurement', var_name='var')


In [ ]:
plot = (
        ggplot(overview, aes('optimizer', y='measurement'))
        + geom_col(width=0.3)
        #+ scale_fill_manual(values=cdict) 
        + facet_wrap('~ wflow + scale', scales='free')
        + ggtitle(plot_title)
        + xlab("Optimization Method")
        + ylab("Time (in sec.)")
        + theme(figure_size=(5, 2.5),
               subplots_adjust={'hspace': 1, 'wspace': 0.25})
    )

In [ ]:
plot.save("specific-plots/5end_to_end.png")
tikzplotlib.save('specific-plots/5end_to_end.tikz', figure=plot)
display(plot)